# Slow waves analysis

## Introductory notes:
This notebook presents slow waves detection functionality.

Recommended reading:
1. [YASA: FAQ](https://raphaelvallat.com/yasa/build/html/faq.html#event-detection)
2. [YASA: Slow-waves detection](https://github.com/raphaelvallat/yasa/blob/master/notebooks/05_sw_detection.ipynb)
3. [YASA: Multi-channel slow-wave detection](https://github.com/raphaelvallat/yasa/blob/master/notebooks/06_sw_detection_multi.ipynb)
4. [YASA: SWResults class](https://raphaelvallat.com/yasa/build/html/generated/yasa.SWResults.html#yasa-swresults)
5. [MNE: Time-frequency analysis: power and inter-trial coherence](https://mne.tools/stable/auto_tutorials/time-freq/20_sensors_time_frequency.html#time-frequency-analysis-power-and-inter-trial-coherence)
6. [MNE: AverageTFR class](https://mne.tools/stable/generated/mne.time_frequency.AverageTFR.html#mne-time-frequency-averagetfr)

## Import module

In [1]:
from sleepeegpy.pipeline import SlowWavesPipe
from joblib import parallel_backend; parallel_backend("loky", n_jobs=-1)
from os import makedirs,path

## Slow waves detection

If you wish to change the path for output_dir ot input dir, change it below.
If no such folders, they will be created automatically.

In [2]:
output_dir  = "output_folder" # Output path and name can be changed here
input_dir = "input_files" # input files dir can be changed here
makedirs(input_dir, exist_ok=True)
makedirs(output_dir, exist_ok=True)

#### Add required files and data
* Put all your files in the input folder.
* Modify your eeg file name below. The file can be any format supported by the mne.read_raw() function.
* Modify your hypnogram file name below
* Make sure the hypno_freq is the right frequency.
* For more information about the supported formats, see [mne documentation](https://mne.tools/stable/generated/mne.io.Raw.html)

In [3]:
eeg_file_name= "resampled_raw.fif" #None # add your eeg_path here
hypnogram_filename = "staging.txt" # Hypnogram filename can be changed here (file must be in the input dir)
hypno_freq = 1

In [4]:
path_to_eeg = path.join(input_dir,eeg_file_name)
hypnogram_path = path.join(input_dir,hypnogram_filename)

assert path.isfile(path_to_eeg) and path.isfile(hypnogram_path), f"{path_to_eeg} or {hypnogram_path} not exist"
slow_waves_pipe = SlowWavesPipe(
    path_to_eeg=path_to_eeg,
    output_dir=output_dir,
    path_to_hypno=hypnogram_path,
    hypno_freq=hypno_freq,
)

Opening raw data file input_files/resampled_raw.fif...
    Range : 0 ... 1994499 =      0.000 ...  7977.996 secs
Ready.
Opening raw data file /Users/yardenmezi/Desktop/myProjects/sleeping_lab/repos/sleepeegpy/notebooks/input_files/resampled_raw-1.fif...
    Range : 1994500 ... 3988999 =   7978.000 ... 15955.996 secs
Ready.
Opening raw data file /Users/yardenmezi/Desktop/myProjects/sleeping_lab/repos/sleepeegpy/notebooks/input_files/resampled_raw-2.fif...
    Range : 3989000 ... 5983499 =  15956.000 ... 23933.996 secs
Ready.
Opening raw data file /Users/yardenmezi/Desktop/myProjects/sleeping_lab/repos/sleepeegpy/notebooks/input_files/resampled_raw-3.fif...
    Range : 5983500 ... 5985607 =  23934.000 ... 23942.428 secs
Ready.


Hypnogram is LONGER than data by 23882.0 seconds. Cropping hypnogram to match data.size.


Please refer to the YASA's [documentation](https://raphaelvallat.com/yasa/build/html/generated/yasa.sw_detect.html) for details about the arguments.

In [5]:
slow_waves_pipe.detect(
    picks=["eeg"],
    reference="average",
    include=(2, 3),
    freq_sw=(0.3, 1.5),
    dur_neg=(0.3, 1.5),
    dur_pos=(0.1, 1),
    amp_neg=(40, 200),
    amp_pos=(10, 150),
    amp_ptp=(75, 350),
    coupling=False,
    coupling_params={"freq_sp": (12, 16), "p": 0.05, "time": 1},
    remove_outliers=False,
    verbose=False,
    save=True,
)

Reading 0 ... 15000  =      0.000 ...    60.000 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


AssertionError: None of the stages specified in `include` are present in hypno.

Please refer to the YASA's [documentation](https://raphaelvallat.com/yasa/build/html/generated/yasa.SWResults.html) for details about the arguments.

In [6]:
slow_waves_pipe.results.summary(
    grp_chan=False, grp_stage=True, mask=None, aggfunc="mean", sort=True
)

AttributeError: 'SlowWavesPipe' object has no attribute 'results'

In [7]:
slow_waves_pipe.plot_average(
    save=True,
    center="NegPeak",
    hue="Stage",
    time_before=0.4,
    time_after=0.8,
    filt=(None, None),
    mask=None,
)

AttributeError: 'SlowWavesPipe' object has no attribute 'results'

In [8]:
slow_waves_pipe.plot_topomap(
    prop="Slope",  #  Any of the event properties returned by pipe.results.summary().
    stage="N3",  # Stage to plot the topomap for.
    aggfunc="mean",  # Aggregation function for channel grouping.
    # Should contain at least index of the provided "stage".
    sleep_stages={"Wake": 0, "N1": 1, "N2": 2, "N3": 3, "REM": 4},
    axis=None,  # Whether to plot on provided matplotlib axis.
    save=True,  # Whether to save the plot as a file.
    topomap_args=dict(cmap="plasma"),  # Arguments passed to mne.viz.plot_topomap().
    cbar_args=None,  # Arguments passed to plt.colorbar().
)

AttributeError: 'SlowWavesPipe' object has no attribute 'results'

In [9]:
slow_waves_pipe.plot_topomap_collage(
    # Some of the event properties returned by pipe.results.summary().
    props=["Slope", "Frequency", "Duration"],
    aggfunc="mean",  # Aggregation function for channel grouping.
    # Tuple of strs or "all", e.g., ("N1", "REM") or "all" (plots all "sleep_stages").
    stages_to_plot="all",
    # Should contain at least indices of the provided "stages_to_plot".
    sleep_stages={"N2": 2, "N3": 3},
    low_percentile=5,  # Set min color value by percentile of the property data.
    high_percentile=95,  # Set max color value by percentile of the property data.
    fig=None,  # Instance of plt.Figure, a new fig will be created if None.
    save=True,  # Whether to save the plot as a file.
    topomap_args=dict(cmap="plasma"),  # Arguments passed to mne.viz.plot_topomap().
    cbar_args=None,  # Arguments passed to plt.colorbar().
)

AttributeError: 'SlowWavesPipe' object has no attribute 'results'

Constructs MNE [AverageTFR](https://mne.tools/stable/generated/mne.time_frequency.AverageTFR.html) object for every available sleep stage.

In [10]:
slow_waves_pipe.compute_tfr(
    freqs=(0.5, 5),  # Interval of frequencies of interest.
    n_freqs=100,  # TFR frequency resolution.
    time_before=4,  # Time in sec before the peak to use in TFR computation. Should be positive.
    time_after=4,  # Time in sec after the peak to use in TFR computation.
    method="morlet",  # TFR method: 'morlet' or 'multitaper'.
    save=True,  # Whether to save AverageTFR object per sleep stage as hdf5 file.
    overwrite=False,  # Whether to overwrite existing files.
    n_cycles=2,
)

slow_waves_pipe.tfrs

AttributeError: 'SlowWavesPipe' object has no attribute 'results'

In [11]:
slow_waves_pipe.tfrs["N2"].data

AttributeError: 'SlowWavesPipe' object has no attribute 'tfrs'

In [12]:
slow_waves_pipe.tfrs["N2"].to_data_frame()

AttributeError: 'SlowWavesPipe' object has no attribute 'tfrs'

In [13]:
slow_waves_pipe.tfrs["N2"].plot(["E101"]),
slow_waves_pipe.tfrs["N3"].plot(["E101"])

AttributeError: 'SlowWavesPipe' object has no attribute 'tfrs'

If you want to load saved TFR files:

In [15]:
slow_waves_pipe.read_tfrs(dirpath=None)